# Initialization

In [ ]:
import bgk

In [ ]:
B = 10
res = 512

path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/double_prt/B1_n512_v+/"
length = {1:(1., .03, .03), 10:(1., .02, .02), .1:(1., .18, .18), 2:(1., .028, .028)}[B]
wholeSlice = bgk.DataSlice(slice(-length[1]/2, length[1]/2), "")
centerSlice = bgk.DataSlice({10:slice(-.001, .001), 1:slice(-.003, .003), 0.1:slice(-.03, .03), 2:slice(-.002, .002)}[B], "Central ")

loader = bgk.Loader(path, length, engine="pscadios2", species_names=['e', 'i'])

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 100

videoMaker = bgk.VideoMaker(nframes, loader)

completion_percent = 100 * loader.fields_max / loader.nmax
video_coverage_percent = 100 * nframes * videoMaker.fields_stepsPerFrame / loader.fields_max
steps_used_percent = 100 * nframes / (loader.fields_max / loader.fields_every)
print(f"steps simulated:      {loader.fields_max} ({completion_percent:.1f}% complete)")
print(f"nframes in animation: {nframes}")
print(f"steps per frame:      {videoMaker.fields_stepsPerFrame}")
print(f"max step in video:    {nframes * videoMaker.fields_stepsPerFrame} ({video_coverage_percent:.1f}% coverage, {steps_used_percent:.1f}% step used)")
if video_coverage_percent != 100:
    print(f"suggested nframes:    {loader.get_all_suggested_nframes(nframes)[0]}")

# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [ ]:
from IPython.display import HTML

In [ ]:
paramId = -1
param = [
        bgk.ParamMetadata('Electron Density', 0, None, 'inferno', 'pfd_moments', 'rho_e', coef=-1),
        bgk.ParamMetadata('Ion Density', 0, None, 'inferno', 'pfd_moments', 'rho_i'),
        bgk.ParamMetadata('Y-ial E', -.0005, .0005, 'RdBu', 'pfd', 'ey_ec'),
        bgk.ParamMetadata('Y-ial B', -3e-8, 3e-8, 'RdBu', 'pfd', 'hy_fc', skipFirst=True),
        bgk.ParamMetadata('X-ial B', -B*1.2, B*1.2, 'RdBu', 'pfd', 'hx_fc'),
        bgk.ParamMetadata('Y-ial J', -.0005, .0005, 'RdBu', 'pfd', 'jy_ec', skipFirst=True),
        bgk.ParamMetadata('Radial J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='radial'),
        bgk.ParamMetadata('Azimuthal J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='azimuthal'),
        bgk.ParamMetadata('Electron Temperature', 0, None, 'inferno', 'pfd_moments', ['tyy_e', 'tzz_e'], combine='sum'),
        bgk.ParamMetadata('Ion Temperature', 0, None, 'inferno', 'pfd_moments', ['tyy_i', 'tzz_i'], combine='sum'),
        bgk.ParamMetadata('Div E', -1.5, 1.5, 'RdBu', 'gauss', 'dive'),
        bgk.ParamMetadata('Charge Density', -1.5, 1.5, 'RdBu', 'gauss', 'rho'),
        bgk.ParamMetadata('Gauss Error', -.005, .005, 'RdBu', 'gauss', ['dive', 'rho'], combine='difference'),
        bgk.ParamMetadata('Radial Electron Velocity', -.0005, .0005, 'RdBu', 'pfd_moments', ['jy_e', 'jz_e'], combine='radial', coef=-1),
        bgk.ParamMetadata('Azimuthal Electron Velocity', -.0005, .0005, 'RdBu', 'pfd_moments', ['jy_e', 'jz_e'], combine='azimuthal', coef=-1),
        bgk.ParamMetadata('Azimuthal Ion Velocity', None, None, 'RdBu', 'pfd_moments', ['jy_i', 'jz_i'], combine='azimuthal'),
        bgk.ParamMetadata('Radial E', None, None, 'RdBu', 'pfd', ['ey_ec', 'ez_ec'], combine='radial'),
        ][paramId]
print(f"quantity: {param.title}")

In [ ]:
# load data
videoMaker.loadData(param)

In [ ]:
# slice data
sliceId = 0
whichSlice = [wholeSlice, centerSlice][sliceId]
videoMaker.setSlice(whichSlice)

print(f"view: {whichSlice.viewAdjective}= {whichSlice.slice}")

In [ ]:
# view t=0
%matplotlib widget
fig, ax, im = videoMaker.viewFrame(0)

In [ ]:
# make movie
%matplotlib widget
anim = videoMaker.viewMovie(fig, ax, im)
HTML(anim.to_html5_video())

## Stability Plot

In [ ]:
%matplotlib widget
videoMaker.viewStability()

# Radial Dependence

In [ ]:
import matplotlib

maxR = max(activeSlice.start, activeSlice.stop) * 2**.5
rStep = length[1] / 100

def getMeanAndStd(data, r):
    rslice = data.where((r <= rGrid) & (rGrid < r + rStep))
    return rslice.mean().item(), rslice.std().item()
quantiles = np.linspace(0, 1, 11)
def getPercentiles(data, r):
    rslice = data.where((r <= rGrid) & (rGrid < r + rStep))
    return tuple(np.nanquantile(rslice.values, p) for p in quantiles)

rs = np.arange(0, maxR, rStep)
def getMeansAndStds(data):
    return tuple(zip(*[getMeanAndStd(data, r) for r in rs]))
def getPercentileses(data):
    return tuple(zip(*[getPercentiles(data, r) for r in rs]))

In [ ]:
dataIdx = -1

### Mean & Error

In [ ]:
means, stds = getMeansAndStds(slicedDatas[dataIdx])
means0, stds0 = getMeansAndStds(slicedDatas[0])

In [ ]:
%matplotlib widget

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
plt.title("Mean " + param.title + " vs Radius (t={:.2f})".format(times[dataIdx]))

plt.errorbar(rs, means0, yerr=stds0, errorevery=(1,2), color="blue", ecolor="lightskyblue", elinewidth=1, capsize=1.5, label="t={:.2f}".format(0))
plt.errorbar(rs, means, yerr=stds, errorevery=(0, 2), color="red", ecolor="lightcoral", elinewidth=1, capsize=1.5, label="t={:.2f}".format(times[dataIdx]))
plt.legend()
plt.show()

### Percentiles

In [ ]:
percentiles = getPercentileses(slicedDatas[dataIdx])

In [ ]:
%matplotlib widget

plt.xlabel("Distance from Axis, $\\rho$")
plt.ylabel(param.title)
plt.title(f"{len(quantiles)} Percentiles of " + param.title + " vs $\\rho$ (t={:.2f})".format(times[dataIdx]))

cmap = matplotlib.cm.get_cmap('rainbow')

labels = ["{:.0f}%".format(q*100) for q in quantiles]
colors = [cmap(2*abs(.5-q)) for q in quantiles]

for perc, color, label in reversed(list(zip(percentiles, colors, labels))):
    plt.plot(rs, perc, color=color, label=label)

# plt.legend()
plt.show()

### Over a Period

### Aggregate

In [ ]:
allMeans = np.array([getMeansAndStds(slicedDatas[idx])[0] for idx in range(nframes)])

In [ ]:
%matplotlib widget

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
plt.title(f"Time-Averaged Radial Dependence of " + param.title)

plt.plot(rs, allMeans.min(axis=0), color="lightcoral", label="min mean")
plt.plot(rs, allMeans.mean(axis=0), color="blue", label="mean mean")
plt.plot(rs, allMeans.max(axis=0), color="lightcoral", label="max mean")
plt.legend()
plt.show()